## Converting pdf to csv

In [40]:
"""
STAGE 1: PDF TO CSV CONVERSION
Extracts M-Pesa statement from PDF and saves as CSV
"""

import tabula
import pandas as pd

def convert_mpesa_pdf_to_csv(pdf_path: str, password: str, output_csv: str):
    """
    Convert M-Pesa PDF statement to CSV
    
    Args:
        pdf_path: Path to M-Pesa PDF statement
        password: PDF password
        output_csv: Output CSV file path
    """
    
    print("=" * 80)
    print("STAGE 1: PDF TO CSV CONVERSION")
    print("=" * 80)
    print()
    
    print(f"📄 Reading PDF: {pdf_path}")
    print(f"🔐 Using password: {'*' * len(password)}")
    print()
    
    # Extract ALL tables from ALL pages with 'latin-1' encoding
    tables = tabula.read_pdf(
        pdf_path, 
        password=password,
        encoding='latin-1',
        pages='all',  # Extract from all pages
        multiple_tables=True  # Get all tables on each page
    )
    
    print(f"✓ Extracted {len(tables)} tables from PDF")
    
    # Combine all tables
    df = pd.concat(tables, ignore_index=True)
    
    print(f"✓ Combined into {len(df)} rows and {len(df.columns)} columns")
    print()
    
    # Save to CSV
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    
    print(f"✅ CSV saved successfully: {output_csv}")
    print()
    
    # Show preview
    print("=" * 80)
    print("PREVIEW - First 5 rows:")
    print("=" * 80)
    print(df.head())
    print()
    
    print("=" * 80)
    print("PREVIEW - Last 5 rows:")
    print("=" * 80)
    print(df.tail())
    print()
    
    print(f"Total rows: {len(df):,}")
    print(f"Total columns: {len(df.columns)}")
    print()
    
    return df


if __name__ == "__main__":
    # CONFIGURE THESE PATHS
    PDF_FILE = r"C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\Mpesa statement.pdf"
    PDF_PASSWORD = "768343"  # Your PDF password
    OUTPUT_CSV = r"C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\stage1_mpesa_raw.csv"
    
    # Run conversion
    df = convert_mpesa_pdf_to_csv(PDF_FILE, PDF_PASSWORD, OUTPUT_CSV)
    
    print("✅ STAGE 1 COMPLETE!")
    print(f"Output: {OUTPUT_CSV}")

STAGE 1: PDF TO CSV CONVERSION

📄 Reading PDF: C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\Mpesa statement.pdf
🔐 Using password: ******



Got stderr: Feb 17, 2026 12:34:10 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Feb 17, 2026 12:34:25 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



✓ Extracted 261 tables from PDF
✓ Combined into 5351 rows and 14 columns

✅ CSV saved successfully: C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\stage1_mpesa_raw.csv

PREVIEW - First 5 rows:
   Unnamed: 0  Unnamed: 1           TRANSACTION TYPE     PAID IN    PAID OUT  \
0         NaN         NaN                SEND MONEY:        0.00  249,502.00   
1         NaN         NaN            RECEIVED MONEY:  506,871.21        0.00   
2         NaN         NaN             AGENT DEPOSIT:    1,700.00        0.00   
3         NaN         NaN          AGENT WITHDRAWAL:        0.00   48,648.00   
4         NaN         NaN  LIPA NA M-PESA (PAYBILL):        0.00  260,647.00   

  Receipt No. Completion Time Details Transaction Status Paid In Withdrawn  \
0         NaN             NaN     NaN                NaN     NaN       NaN   
1         NaN             NaN     NaN                NaN     NaN       NaN   
2         NaN             NaN     NaN                NaN     NaN       NaN   
3         N

## Key Categorization

In [44]:
"""
STAGE 2: TRANSACTION TYPE IDENTIFICATION (COMPLETE FIX)
✅ Separates Fuliza LOAN (OverDraft of Credit Party) from Fuliza PAYMENTS
✅ Loan Repayment as separate category
✅ All other fixes included
"""

import pandas as pd
import re
from typing import Dict


class TransactionTypeIdentifier:
    """Enhanced transaction type identification with all fixes"""
    
    def __init__(self):
        """Define patterns in strict priority order"""
        # Format: (type_name, [patterns], priority)
        self.type_patterns = [
            # PRIORITY 1: Fees (check first - often confused with other types)
            ('M-Pesa Fee', [
                r'transfer\s+of\s+funds\s+charge',
                r'pay\s+bill\s+charge',
                r'pay\s+merchant\s+charge',
                r'withdraw(al)?\s+charge',
                r'\bcharge\b$',
            ], 1),
            
            # PRIORITY 2: Fuliza/Overdraft LOAN (the credit itself - OverDraft of Credit Party)
            ('Fuliza', [
                r'overdraft\s+of\s+credit\s+party',  # This is the LOAN
            ], 2),
            
            # PRIORITY 3: Loan Repayment (paying back loans, including Fuliza payments)
            ('Loan Repayment', [
                r'od\s+loan\s+repayment',
                r'loan\s+repayment',
                r'fuliza\s+repayment',
                r'overdraw',
            ], 3),
            
            # PRIORITY 4: LOOP Payment (Income from LOOP)
            ('LOOP Payment', [
                r'promotion\s+payment\s+from.*loop\s+b2c',
                r'loop\s+b2c',
            ], 4),
            
            # PRIORITY 5: Received Money (Income)
            ('Received Money', [
                r'funds\s+received\s+from',
                r'business\s+payment\s+from',
                r'received\s+from',
            ], 5),
            
            # PRIORITY 6: Cash Deposit (at agent)
            ('Cash Deposit', [
                r'deposit\s+of\s+funds\s+at\s+agent',
            ], 6),
            
            # PRIORITY 7: Cash Withdrawal (at agent)
            ('Cash Withdrawal', [
                r'customer\s+withdrawal\s+at\s+agent',
                r'withdrawal\s+at\s+agent',
            ], 7),
            
            # PRIORITY 8: Data Bundles (separated from Airtime)
            # Including Fuliza-powered data bundles
            ('Data Bundles', [
                r'safaricom\s+data',
                r'safaricom\s+data\s+bundles',
                r'customer\s+bundle\s+purchase\s+with\s+fuliza.*4093441',
                r'(?i)buy\s+bundle',
                r'(?i)customer\s+bundle\s+purchase',
                 r'customer\s+bundle\s+purchase\s+with\s+fuliza',
            
            ], 8),
            
            # PRIORITY 9: Airtime (separated from Data, includes Direct Pay)
            # Including Fuliza-powered airtime
            ('Airtime', [
                r'(?i)safaricom\s+offers',  # Safaricom Offers = Airtime
                r'airtime\s+purchase',
                r'pay\s+bill.*direct\s+pay.*atl\d+',  # Direct Pay airtime
                r'4187661.*direct\s+pay',  # Direct Pay paybill
                r'4093275.*direct\s+pay',  # Another Direct Pay paybill
                r'recharge\s+for\s+customer',
            ], 9),
            
            # PRIORITY 10: Send Money (including Fuliza-powered transfers)
            ('Send Money', [
                r'(?i)customer\s+transfer\s+to\s+-\s+(2547|07|01)[\d\*]+',
                r'customer\s+transfer\s+to\s+-\s+',
                r'(?i)customer\stransfer',
                r'customer\s+send\s+money.*fuliza.*to\s+-\s+(2547|07|01)[\d\*]+',
                r'(?i)customer\s+transfer\s+fuliza\s+mpesa\s*to\s+-\s+(2547|07|01)[\d\*]+',
            ], 10),
            
            # PRIORITY 11: Pochi la Biashara
            ('Pochi la Biashara', [
                r'customer\s+payment\s+to\s+small\s+business',
            ], 11),
            
            # PRIORITY 12: Till Payment (including Fuliza-powered)
            ('Till Payment', [
                r'merchant\s+payment\s+(online\s+)?to\s+\d+',
                r'merchant\s+payment\s+fuliza\s+m-?pesa\s*to\s+\d+',
                r'till\s+\d+',
            ], 12),
            
            # PRIORITY 13: PayBill (including Fuliza-powered)
            ('PayBill', [
                r'pay\s+bill\s+(online\s+)?to\s+\d+',
                r'pay\s+bill\s+fuliza\s+m-?pesa\s+to\s+\d+',
                r'pay\s+bill\s+online\s+fuliza\s+m-pesa\s+to\s+(\d+)\s+-\s+([\w\s]+?)\s+acc\.?\s+([\w\s]+)',
            ], 13),
            
            # PRIORITY 14: M-Shwari
            ('M-Shwari', [
                r'm-?\s*shwari',
            ], 14),
            
            # PRIORITY 15: Unit Trust
            ('Unit Trust', [
                r'unit\s+trust',
                r'ziidi',
            ], 15),
            
            # PRIORITY 16: Reversal
            ('Reversal', [
                r'reversal',
                r'reversed',
            ], 16),
        ]
    
    def identify_type(self, description: str) -> str:
        """Identify transaction type"""
        if pd.isna(description) or description == '':
            return 'Other'
        
        desc_lower = str(description).lower().strip()
        
        # Check in priority order
        for trans_type, patterns, _ in self.type_patterns:
            for pattern in patterns:
                if re.search(pattern, desc_lower, re.IGNORECASE):
                    return trans_type
        
        return 'Other'
    
    def extract_fields(self, description: str, txn_type: str) -> Dict:
        """Extract key fields from description"""
        if pd.isna(description):
            return {}
        
        fields = {}
        desc = str(description)
        
        if txn_type == "Send Money":
            # Regular transfer
            match = re.search(
                r'(?i)customer\s+transfer\s+(?:fuliza\s+mpesa\s*)?to\s+-\s+((2547|07|01)[\d\*]+)\s+(.*)',
                desc
            )
            if match:
                fields["recipient_number"] = match.group(1)
                fields["recipient_name"] = match.group(3).strip()
        
        elif txn_type == "Pochi la Biashara":
            match = re.search(
                r'(?i)small\s+business\s+to\s+-\s+((2547|07|01)[\d\*]+)\s+(.*)',
                desc
            )
            if match:
                fields["recipient_number"] = match.group(1)
                fields["recipient_name"] = match.group(3).strip()
        
        elif txn_type == "Till Payment":
            # Regular or Fuliza merchant payment
            match = re.search(
                r'(?i)merchant\s+payment\s+(?:fuliza\s+m-?pesa\s*)?(?:online\s+)?to\s+(\d+)\s+-\s+(.*)',
                desc
            )
            if match:
                fields["till_number"] = match.group(1)
                raw_merchant = match.group(2).strip()
                raw_merchant = re.sub(
                    r'(?i)\s+via\s+(coop|equity|kcb|ncba|family)\s+bank\.?$',
                    '', raw_merchant
                ).strip()
                fields["merchant_name"] = raw_merchant
        
        elif txn_type == "PayBill":
            # Regular or Fuliza paybill
            match = re.search(
                r'(?i)pay\s+bill\s+(?:fuliza\s+m-?pesa\s*)?(?:online\s+)?to\s+(\d+)\s+[-–]\s+([\w\s]+?)\s+[Aa]cc\.?\s+([\w#]+)',
                desc
            )
            if match:
                fields["paybill_number"] = match.group(1)
                fields["merchant_name"] = match.group(2).strip()
                fields["account_number"] = match.group(3).strip()
            else:
                match2 = re.search(
                    r'(?i)pay\s+bill\s+(?:fuliza\s+m-?pesa\s*)?(?:online\s+)?to\s+(\d+)\s+[-–]?\s+(.*)',
                    desc
                )
                if match2:
                    fields["paybill_number"] = match2.group(1)
                    fields["merchant_name"] = match2.group(2).strip()
        
        elif txn_type in ["Cash Withdrawal", "Cash Deposit"]:
            match = re.search(
                r'(?i)agent\s+till\s+(\d+)\s+[-–]\s+(.*)',
                desc
            )
            if match:
                fields["agent_till"] = match.group(1)
                fields["agent_name"] = match.group(2).strip()
        
        elif txn_type in ["Received Money", "LOOP Payment"]:
            match = re.search(
                r'(?i)(?:funds\s+received|payment)\s+from\s+[-–]?\s+(\d+)\s+[-–]\s+(.*)',
                desc
            )
            if match:
                fields["sender_number"] = match.group(1)
                fields["sender_name"] = match.group(2).strip()
        
        return fields
    
    def process_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add transaction_type and extracted_fields"""
        print("🔍 Identifying transaction types...")
        
        # Clean description
        df['description_clean'] = df['Details'].apply(self._clean_text)
        
        # Identify types
        df['transaction_type'] = df['description_clean'].apply(self.identify_type)
        
        # Extract fields
        print("📋 Extracting details...")
        df['extracted_fields'] = df.apply(
            lambda row: self.extract_fields(row['description_clean'], row['transaction_type']),
            axis=1
        )
        
        print(f"✓ Identified {df['transaction_type'].nunique()} transaction types")
        
        return df
    
    def _clean_text(self, text: str) -> str:
        """Clean multiline PDF text"""
        if pd.isna(text):
            return ''
        text = str(text).replace('\\r', ' ').replace('\\n', ' ').replace('\r', ' ').replace('\n', ' ')
        text = re.sub(r'\s+', ' ', text)
        return text.strip()


def run_stage2(input_csv: str, output_csv: str):
    """Run Stage 2"""
    
    print("=" * 80)
    print("STAGE 2: TRANSACTION TYPE IDENTIFICATION (COMPLETE FIX)")
    print("=" * 80)
    print()
    print("✅ OverDraft of Credit Party → Fuliza (LOAN)")
    print("✅ Fuliza payments → Categorized by what was paid (Airtime, Till, etc.)")
    print("✅ Loan Repayment separate category")
    print("✅ LOOP B2C → Income")
    print("✅ Data Bundles (4093441) ≠ Airtime (826915)")
    print("✅ Direct Pay → Airtime")
    print("✅ Deposit/Withdrawal at Agent → separate")
    print()
    
    # Load
    print(f"📂 Loading: {input_csv}")
    df = pd.read_csv(input_csv, low_memory=False)
    
    # Filter to transactions
    mask = df['Receipt No.'].notna() & (df['Receipt No.'] != '')
    df = df[mask].copy()
    
    print(f"✓ Loaded {len(df)} transactions")
    print()
    
    # Process
    identifier = TransactionTypeIdentifier()
    df = identifier.process_dataframe(df)
    
    print()
    print("=" * 80)
    print("TRANSACTION TYPE BREAKDOWN")
    print("=" * 80)
    for trans_type, count in df['transaction_type'].value_counts().items():
        pct = (count / len(df)) * 100
        print(f"{trans_type:25s}: {count:5d} ({pct:5.1f}%)")
    
    print()
    print("=" * 80)
    print("VERIFICATION - KEY TYPES")
    print("=" * 80)
    
    # Verify critical fixes
    key_types = ['Fuliza', 'Loan Repayment', 'LOOP Payment', 'Data Bundles', 
                 'Airtime', 'Cash Deposit', 'Cash Withdrawal']
    
    for trans_type in key_types:
        type_df = df[df['transaction_type'] == trans_type]
        if len(type_df) > 0:
            print(f"\n{trans_type} ({len(type_df)} transactions):")
            for _, row in type_df.head(2).iterrows():
                print(f"  {row['description_clean'][:70]}")
    
    print()
    
    # Show Fuliza-powered transactions
    fuliza_powered = df[df['description_clean'].str.contains('fuliza', case=False, na=False)]
    print("=" * 80)
    print(f"FULIZA-POWERED TRANSACTIONS: {len(fuliza_powered)} total")
    print("=" * 80)
    fuliza_breakdown = fuliza_powered['transaction_type'].value_counts()
    for txn_type, count in fuliza_breakdown.items():
        print(f"  {txn_type:25s}: {count:5d}")
    print()
    print("✓ Fuliza LOAN (OverDraft) vs Fuliza PAYMENTS properly separated!")
    print()
    
    # Save
    df['extracted_fields_str'] = df['extracted_fields'].apply(str)
    df.to_csv(output_csv, index=False)
    print(f"✅ Saved: {output_csv}")
    print()
    
    return df


if __name__ == "__main__":
    INPUT = r"C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\stage1_mpesa_raw.csv"
    OUTPUT = r"C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\stage2_with_types.csv"
    
    df = run_stage2(INPUT, OUTPUT)
    print("✅ Ready for Stage 3")

STAGE 2: TRANSACTION TYPE IDENTIFICATION (COMPLETE FIX)

✅ OverDraft of Credit Party → Fuliza (LOAN)
✅ Fuliza payments → Categorized by what was paid (Airtime, Till, etc.)
✅ Loan Repayment separate category
✅ LOOP B2C → Income
✅ Data Bundles (4093441) ≠ Airtime (826915)
✅ Direct Pay → Airtime
✅ Deposit/Withdrawal at Agent → separate

📂 Loading: C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\stage1_mpesa_raw.csv
✓ Loaded 5029 transactions

🔍 Identifying transaction types...
📋 Extracting details...
✓ Identified 17 transaction types

TRANSACTION TYPE BREAKDOWN
Send Money               :  1189 ( 23.6%)
M-Pesa Fee               :   606 ( 12.1%)
Till Payment             :   534 ( 10.6%)
Fuliza                   :   428 (  8.5%)
Received Money           :   408 (  8.1%)
Pochi la Biashara        :   322 (  6.4%)
M-Shwari                 :   314 (  6.2%)
Airtime                  :   311 (  6.2%)
Data Bundles             :   297 (  5.9%)
PayBill                  :   218 (  4.3%)
Loan Repaymen

## Transansaction Type Identification

In [45]:
"""
STAGE 3: KEYWORD-BASED CATEGORIZATION (COMPLETE FIX)
All issues addressed:
✅ Removed "online" and "online purchase" from Online Shopping
✅ Removed "fuliza" from Loans (handled in Stage 2)
✅ Fuliza → Loans (via transaction type)
✅ LOOP Payment → Income
✅ Data Bundles and Airtime now separate
✅ Direct Pay airtime not miscategorized as Online Shopping
"""

import pandas as pd
import re
from typing import Dict


class KeywordCategorizer:
    """Enhanced keyword categorizer with all fixes"""
    
    def __init__(self):
        self.category_keywords = {
            # HIGH PRIORITY
            'Health Care': {
                'keywords': [
                    'hospital', 'clinic', 'pharmacy', 'medical', 'nhif',
                    'chemist', 'doctor', 'laboratory', 'lab', 'diagnostic',
                    'aga khan', 'nairobi hospital', 'mater', 'kenyatta hospital',
                    'mp shah', 'gertrudes', 'lancet', 'dental', 'optical',
                ],
                'priority': 1,
            },
            
            'Betting': {
                'keywords': [
                    'sportpesa', 'sportybet', 'betika', '1xbet', 'stake', 
                    'bangbet', '22bet', 'mozzart bet', 'betway', 'odibets',
                    'kareco holdings', 'melbet', 'betin', 'betpawa', 'shabiki',
                    'bet', 'betting', 'lotto', 'lottery', 'casino',
                ],
                'priority': 1,
            },
            
            # Loans - REMOVED "fuliza" (now handled via transaction type)
            'Loans': {
                'keywords': [
                    'm-shwari loan', 'kcb m-pesa loan', 'hustler fund',
                    'okash', 'zenka', 'timiza', 'Overdraft',
                ],
                'priority': 1,
            },

            'Loan Repayment': {
                'keywords': ['repayment', 'overdraw'],
                'priority': 1,
            },
            
            # Online Shopping - REMOVED "online" and "online purchase"
            'Online Shopping': {
                'keywords': [
                    'jumia', 'kilimall', 'masoko', 'glovo', 'jiji',
                    'aliexpress', 'amazon', 'alibaba', 'uber eats', 'bolt food',
                    'sky garden', 'food delivery', 'home delivery',
                ],
                'priority': 1,
            },
            
            'Bills': {
                'keywords': [
                    'kplc', 'water', 'rent', 'insurance', 'gas refill',
                    'internet', 'home wifi', 'land rates', 'security',
                    'parking', 'electricity', 'prepaid', 'postpaid',
                ],
                'priority': 1,
            },
            
            'Subscriptions': {
                'keywords': [
                    'netflix', 'spotify', 'youtube', 'prime', 'hbo',
                    'gotv', 'dstv', 'showmax', 'apple music', 'startimes',
                    'zuku', 'subscription', 'microsoft 365', 'office 365',
                ],
                'priority': 1,
            },
            
            'Education': {
                'keywords': [
                    'university', 'school', 'college', 'helb', 'kuccps',
                    'knec', 'tvet', 'kmtc', 'fees', 'tuition', 'catering',
                    'kabarak', 'student', 'academy', 'exam fee', 'hostel',
                ],
                'priority': 1,
            },
            
            'Savings': {
                'keywords': [
                    'mshwari deposit', 'unit trust', 'mmf', 'fixed deposit',
                    'investment', 'koala', 'ndovu', 'etica', 'chama',
                    'ziidi', 'savings', 'sacco deposit', 'Sacco'
                ],
                'priority': 1,
            },
            
            # MEDIUM PRIORITY
            'Shopping': {
                'keywords': [
                    'supermarket', 'naivas', 'quickmart', 'quick mart',
                    'carrefour', 'chandarana', 'foodplus', 'cleanshelf',
                    'eastmatt', 'tuskys', 'kabsmart', 'nakumatt', 'Store', 
                ],
                'priority': 2,
            },
            
            'Fast Foods': {
                'keywords': [
                    'kfc', 'chicken inn', 'java house', 'artcaffe',
                    'pizza', 'burger king', 'dominos', 'debonairs',
                    'pizza hut', 'pizza inn', 'subway', 'steers','inn',
                ],
                'priority': 2,
            },
            
            'Food & Dining': {
                'keywords': [
                    'restaurant', 'hotel', 'cafe', 'eatery', 'food court',
                    'dining', 'meat', 'vegetables', 'fruits', 'milk','food',
                ],
                'priority': 2,
            },
            
            'Personal Care': {
                'keywords': [
                    'beauty', 'cosmetics', 'skincare', 'makeup', 'barber',
                    'salon', 'spa', 'kinyozi', 'grooming', 'hair', 'nails',
                ],
                'priority': 2,
            },
            
            'Transport': {
                'keywords': [
                    'uber', 'bolt', 'taxi', 'little cab', 'transport',
                    'fuel', 'petrol', 'diesel', 'shell', 'total', 'parking',
                ],
                'priority': 2,
            },
            
            'Entertainment': {
                'keywords': [
                    'liquor', 'bar', 'wine', 'beer', 'club', 'lounge',
                    'pub', 'cinema', 'bowling', 'arcade', 'entertainment',
                ],
                'priority': 2,
            },
            
            # LOW PRIORITY
            'Bank Transfer': {
                'keywords': [
                    'equity', 'kcb', 'family bank', 'co-op', 'ncba',
                    'stanbic', 'absa', 'bank transfer',
                ],
                'priority': 3,
            },
        }
    
    def categorize(self, description: str, transaction_type: str, extracted_fields: Dict = None) -> str:
        """Categorize transaction"""
        if pd.isna(description):
            return 'Uncategorized'
        
        desc_lower = str(description).lower()
        
        # Build search text
        search_text = desc_lower
        if extracted_fields:
            for key in ['merchant_name', 'recipient_name', 'sender_name', 'agent_name']:
                if key in extracted_fields:
                    search_text += ' ' + str(extracted_fields[key]).lower()
        
        # PRIORITY 1: Transaction type based (FIXED)
        
        # Income
        if transaction_type in ['Received Money', 'LOOP Payment']:
            return 'Income'
        
        # Cash operations
        if transaction_type == 'Cash Deposit':
            return 'Cash Deposit'
        
        if transaction_type == 'Cash Withdrawal':
            return 'Cash Withdrawal'
        
        # Loans (including Fuliza/OverDraft from Stage 2)
        if transaction_type == 'Overdraft':
            return 'Loans'
        
        # Data vs Airtime (now separated in Stage 2)
        if transaction_type == 'Data Bundles':
            return 'Data Bundles'
        
        if transaction_type == 'Airtime':
            return 'Airtime'
        
        # Fees
        if transaction_type == 'M-Pesa Fee':
            return 'M-Pesa Fees'
        
        # M-Shwari
        if transaction_type == 'M-Shwari':
            if 'withdraw' in desc_lower:
                return 'Cash Withdrawal'
            else:
                return 'Savings'
        
        # Unit Trust
        if transaction_type == 'Unit Trust':
            return 'Savings'
        
        # Reversal
        if transaction_type == 'Reversal':
            return 'Reversal'
        
        # PRIORITY 2: Send Money - LEAVE UNCATEGORIZED for Stage 4
        if transaction_type == 'Send Money':
            return 'Uncategorized'
        
        # PRIORITY 3: Till/PayBill/Pochi - Try keywords, fallback to Merchant
        if transaction_type in ['Till Payment', 'PayBill', 'Pochi la Biashara']:
            matched = self._match_keywords(search_text)
            return matched if matched else 'Merchant'
        
        # PRIORITY 4: Other - Try keywords
        matched = self._match_keywords(search_text)
        return matched if matched else 'Other'
    
    def _match_keywords(self, search_text: str) -> str:
        """Match keywords"""
        sorted_categories = sorted(
            self.category_keywords.items(),
            key=lambda x: x[1].get('priority', 99)
        )
        
        for category, rules in sorted_categories:
            for keyword in rules.get('keywords', []):
                pattern = r'\b' + re.escape(keyword) + r'\b'
                if re.search(pattern, search_text, re.IGNORECASE):
                    return category
        
        return ''
    
    def process_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add category column"""
        print("🏷️  Categorizing...")
        
        # Parse extracted_fields
        if 'extracted_fields' in df.columns:
            try:
                import ast
                df['extracted_fields_dict'] = df['extracted_fields'].apply(
                    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.strip() else {}
                )
            except:
                df['extracted_fields_dict'] = df['extracted_fields']
        else:
            df['extracted_fields_dict'] = [{}] * len(df)
        
        # Categorize
        df['category'] = df.apply(
            lambda row: self.categorize(
                row['description_clean'], 
                row['transaction_type'],
                row.get('extracted_fields_dict', {})
            ),
            axis=1
        )
        
        print(f"✓ Categorized into {df['category'].nunique()} categories")
        
        return df


def run_stage3(input_csv: str, output_csv: str):
    """Run Stage 3"""
    
    print("=" * 80)
    print("STAGE 3: KEYWORD-BASED CATEGORIZATION (COMPLETE FIX)")
    print("=" * 80)
    print()
    print("✅ Removed 'online' from Online Shopping")
    print("✅ Removed 'fuliza' from Loans")
    print("✅ Fuliza → Loans (via transaction type)")
    print("✅ LOOP Payment → Income")
    print("✅ Data Bundles & Airtime separate")
    print()
    
    # Load
    print(f"📂 Loading: {input_csv}")
    df = pd.read_csv(input_csv, low_memory=False)
    print(f"✓ Loaded {len(df)} transactions")
    print()
    
    # Categorize
    categorizer = KeywordCategorizer()
    df = categorizer.process_dataframe(df)
    
    print()
    print("=" * 80)
    print("CATEGORY BREAKDOWN")
    print("=" * 80)
    for category, count in df['category'].value_counts().items():
        pct = (count / len(df)) * 100
        print(f"{category:30s}: {count:5d} ({pct:5.1f}%)")
    
    print()
    
    # Verify key categories
    print("=" * 80)
    print("VERIFICATION - KEY CATEGORIES")
    print("=" * 80)
    
    key_cats = ['Loans', 'Data Bundles', 'Airtime', 'Income', 
                'Cash Deposit', 'Cash Withdrawal', 'Loan repayment',]
    
    for cat in key_cats:
        cat_df = df[df['category'] == cat]
        if len(cat_df) > 0:
            print(f"\n{cat} ({len(cat_df)} transactions):")
            for _, row in cat_df.head(2).iterrows():
                print(f"  {row['description_clean'][:70]}")
    
    print()
    
    # Save
    df.to_csv(output_csv, index=False)
    print(f"✅ Saved: {output_csv}")
    print()
    
    return df


if __name__ == "__main__":
    INPUT = r"C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\stage2_with_types.csv"
    OUTPUT = r"C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\stage3_with_categories.csv"
    
    df = run_stage3(INPUT, OUTPUT)
    print("✅ Ready for Stage 4")

STAGE 3: KEYWORD-BASED CATEGORIZATION (COMPLETE FIX)

✅ Removed 'online' from Online Shopping
✅ Removed 'fuliza' from Loans
✅ Fuliza → Loans (via transaction type)
✅ LOOP Payment → Income
✅ Data Bundles & Airtime separate

📂 Loading: C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\stage2_with_types.csv
✓ Loaded 5029 transactions

🏷️  Categorizing...
✓ Categorized into 22 categories

CATEGORY BREAKDOWN
Uncategorized                 :  1329 ( 26.4%)
Merchant                      :   724 ( 14.4%)
M-Pesa Fees                   :   606 ( 12.1%)
Loans                         :   428 (  8.5%)
Income                        :   411 (  8.2%)
Airtime                       :   311 (  6.2%)
Data Bundles                  :   297 (  5.9%)
Cash Withdrawal               :   275 (  5.5%)
Loan Repayment                :   176 (  3.5%)
Savings                       :   118 (  2.3%)
Shopping                      :   101 (  2.0%)
Bank Transfer                 :    94 (  1.9%)
Bills                        

## Smart Rule

In [46]:
"""
STAGE 4: SEND MONEY CATEGORIZATION
Categorizes uncategorized Send Money transactions based on amount and recurring patterns

RULES:
1. Send Money + Recurring (≥2 times) + Amount > 500 → Friends & Family
2. Send Money + Recurring (≥2 times) + Amount ≤ 500 → Merchant
3. Send Money + Non-recurring (any amount) → Merchant

All other categories from Stage 3 remain unchanged.
All improvements from Stage 2 & 3 are preserved:
✅ Fuliza (LOAN) vs Fuliza payments separated
✅ Loan Repayment as separate category
✅ LOOP Payment → Income
✅ Data Bundles ≠ Airtime
✅ Cash Deposit ≠ Cash Withdrawal
"""

import pandas as pd
import re
from typing import Dict
from collections import Counter


class SendMoneyCategorizer:
    """Categorize Send Money transactions using recurring detection and amount thresholds"""
    
    def __init__(self, amount_threshold: float = 500.0, recurring_threshold: int = 2):
        """
        Initialize with thresholds
        
        Args:
            amount_threshold: Amount threshold (default 500 KES)
            recurring_threshold: Minimum occurrences to be recurring (default 2)
        """
        self.amount_threshold = amount_threshold
        self.recurring_threshold = recurring_threshold
    
    def extract_recipient_id(self, extracted_fields_str: str) -> str:
        """
        Extract recipient identifier from extracted_fields string
        
        Args:
            extracted_fields_str: String representation of extracted fields
            
        Returns:
            Recipient identifier (phone number or name)
        """
        if pd.isna(extracted_fields_str) or extracted_fields_str == '':
            return None
        
        try:
            import ast
            fields = ast.literal_eval(extracted_fields_str)
            
            # Use phone number as primary identifier
            if 'recipient_number' in fields:
                return fields['recipient_number']
            elif 'recipient_name' in fields:
                return fields['recipient_name']
        except:
            pass
        
        return None
    
    def detect_recurring_recipients(self, df: pd.DataFrame) -> Dict[str, int]:
        """
        Detect recurring recipients in Send Money transactions
        
        Args:
            df: DataFrame with Send Money transactions
            
        Returns:
            Dictionary mapping recipient IDs to occurrence count
        """
        print("🔍 Detecting recurring recipients in Send Money transactions...")
        
        # Filter to uncategorized Send Money only
        send_money_df = df[
            (df['transaction_type'] == 'Send Money') & 
            (df['category'] == 'Uncategorized')
        ].copy()
        
        if len(send_money_df) == 0:
            print("  No uncategorized Send Money transactions found")
            return {}
        
        # Extract recipient IDs
        send_money_df['recipient_id'] = send_money_df['extracted_fields_str'].apply(
            self.extract_recipient_id
        )
        
        # Count occurrences
        recipient_counts = send_money_df['recipient_id'].value_counts().to_dict()
        
        # Remove None
        recipient_counts = {k: v for k, v in recipient_counts.items() if k is not None}
        
        # Filter to recurring only
        recurring_recipients = {
            k: v for k, v in recipient_counts.items() 
            if v >= self.recurring_threshold
        }
        
        print(f"✓ Found {len(recipient_counts)} unique recipients")
        print(f"✓ {len(recurring_recipients)} recurring recipients (≥{self.recurring_threshold} transactions)")
        
        return recurring_recipients
    
    def categorize_send_money(self, row: pd.Series, recurring_recipients: Dict) -> str:
        """
        Categorize a single Send Money transaction
        
        Args:
            row: DataFrame row
            recurring_recipients: Dict of recurring recipient IDs
            
        Returns:
            Category (Friends & Family or Merchant)
        """
        # Extract recipient ID
        recipient_id = self.extract_recipient_id(row['extracted_fields_str'])
        
        # Check if recurring
        is_recurring = recipient_id in recurring_recipients if recipient_id else False
        
        # Get amount from Withdrawn column
        amount = 0
        if pd.notna(row['Withdrawn']):
            try:
                amount = abs(float(row['Withdrawn']))
            except:
                amount = 0
        
        # Apply rules
        if is_recurring:
            if amount > self.amount_threshold:
                # RULE 1: Recurring + >500 → Friends & Family
                return 'Friends & Family'
            else:
                # RULE 2: Recurring + ≤500 → Merchant
                return 'Merchant'
        else:
            # RULE 3: Non-recurring (any amount) → Merchant
            return 'Merchant'
    
    def process_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Process entire dataframe - categorize Send Money transactions only
        
        Args:
            df: DataFrame with category column
            
        Returns:
            DataFrame with updated categories
        """
        print("🤖 Categorizing Send Money transactions...")
        print()
        
        # Detect recurring recipients
        recurring_recipients = self.detect_recurring_recipients(df)
        
        if recurring_recipients:
            print()
            print("Top 15 recurring recipients:")
            sorted_recipients = sorted(
                recurring_recipients.items(), 
                key=lambda x: x[1], 
                reverse=True
            )[:15]
            for recipient, count in sorted_recipients:
                print(f"  {recipient[:45]:45s}: {count:3d} times")
        
        print()
        
        # Count before
        before_uncat = len(df[
            (df['transaction_type'] == 'Send Money') & 
            (df['category'] == 'Uncategorized')
        ])
        
        # Apply categorization only to uncategorized Send Money
        mask = (df['transaction_type'] == 'Send Money') & (df['category'] == 'Uncategorized')
        
        df.loc[mask, 'category'] = df[mask].apply(
            lambda row: self.categorize_send_money(row, recurring_recipients),
            axis=1
        )
        
        # Count after
        after_uncat = len(df[
            (df['transaction_type'] == 'Send Money') & 
            (df['category'] == 'Uncategorized')
        ])
        family_friends = len(df[
            (df['transaction_type'] == 'Send Money') & 
            (df['category'] == 'Friends & Family')
        ])
        merchant = len(df[
            (df['transaction_type'] == 'Send Money') & 
            (df['category'] == 'Merchant')
        ])
        
        print("=" * 80)
        print("SEND MONEY CATEGORIZATION RESULTS")
        print("=" * 80)
        print(f"Processed: {before_uncat:,} Send Money transactions")
        print(f"  → Friends & Family: {family_friends:,}")
        print(f"  → Merchant: {merchant:,}")
        print(f"  → Still Uncategorized: {after_uncat:,}")
        print()
        
        return df


def run_stage4(input_csv: str, output_csv: str, 
               amount_threshold: float = 500.0, 
               recurring_threshold: int = 5):
    """
    Run Stage 4: Send Money Categorization
    
    Args:
        input_csv: Path to Stage 3 output CSV
        output_csv: Path to save final categorized CSV
        amount_threshold: Amount threshold in KES (default 500)
        recurring_threshold: Min occurrences for recurring (default 2)
    """
    
    print("=" * 80)
    print("STAGE 4: SEND MONEY CATEGORIZATION")
    print("=" * 80)
    print()
    print("Preserving all Stage 2 & 3 improvements:")
    print("  ✅ Fuliza (LOAN) vs Fuliza payments")
    print("  ✅ Loan Repayment separate")
    print("  ✅ LOOP Payment → Income")
    print("  ✅ Data Bundles ≠ Airtime")
    print("  ✅ Cash Deposit ≠ Cash Withdrawal")
    print()
    print("RULES:")
    print(f"  1. Recurring (≥{recurring_threshold}) + Amount > {amount_threshold} → Friends & Family")
    print(f"  2. Recurring (≥{recurring_threshold}) + Amount ≤ {amount_threshold} → Merchant")
    print(f"  3. Non-recurring (any amount) → Merchant")
    print()
    
    # Load data
    print(f"📂 Loading: {input_csv}")
    df = pd.read_csv(input_csv, low_memory=False)
    print(f"✓ Loaded {len(df):,} transactions")
    print()
    
    # Show current state
    print("Current categorization:")
    category_counts = df['category'].value_counts()
    for category in list(category_counts.head(10).index):
        count = category_counts.get(category, 0)
        pct = (count / len(df)) * 100 if len(df) > 0 else 0
        print(f"  {category:25s}: {count:5,} ({pct:5.1f}%)")
    
    send_money_uncat = len(df[
        (df['transaction_type'] == 'Send Money') & 
        (df['category'] == 'Uncategorized')
    ])
    print(f"\n  Send Money (Uncategorized): {send_money_uncat:,}")
    print()
    
    # Process
    categorizer = SendMoneyCategorizer(
        amount_threshold=amount_threshold,
        recurring_threshold=recurring_threshold
    )
    df = categorizer.process_dataframe(df)
    
    # Final summary
    print("=" * 80)
    print("FINAL CATEGORY BREAKDOWN")
    print("=" * 80)
    
    category_counts = df['category'].value_counts().sort_values(ascending=False)
    for category, count in category_counts.items():
        pct = (count / len(df)) * 100
        print(f"{category:30s}: {count:6,} ({pct:5.1f}%)")
    
    print()
    
    # Uncategorized check
    final_uncat = len(df[df['category'] == 'Uncategorized'])
    final_uncat_pct = (final_uncat / len(df)) * 100
    
    print("=" * 80)
    print("CATEGORIZATION SUCCESS RATE")
    print("=" * 80)
    print(f"Total transactions: {len(df):,}")
    print(f"Categorized: {len(df) - final_uncat:,} ({100 - final_uncat_pct:.1f}%)")
    print(f"Uncategorized: {final_uncat:,} ({final_uncat_pct:.1f}%)")
    print()
    
    # Sample results
    print("=" * 80)
    print("SAMPLE CATEGORIZATIONS")
    print("=" * 80)
    
    # Friends & Family samples
    ff_df = df[
        (df['category'] == 'Friends & Family') & 
        (df['transaction_type'] == 'Send Money')
    ]
    if len(ff_df) > 0:
        print(f"\n--- FRIENDS & FAMILY ({len(ff_df):,} transactions) ---")
        for _, row in ff_df.head(5).iterrows():
            try:
                amount = float(row['Withdrawn']) if pd.notna(row['Withdrawn']) else 0.0
            except:
                amount = 0.0
            desc = row['description_clean'][:55] if pd.notna(row['description_clean']) else ''
            print(f"  KES {amount:>8,.0f} | {desc}")
    
    # Merchant samples from Send Money
    merchant_df = df[
        (df['category'] == 'Merchant') & 
        (df['transaction_type'] == 'Send Money')
    ]
    if len(merchant_df) > 0:
        print(f"\n--- MERCHANT - from Send Money ({len(merchant_df):,} transactions) ---")
        for _, row in merchant_df.head(5).iterrows():
            try:
                amount = float(row['Withdrawn']) if pd.notna(row['Withdrawn']) else 0.0
            except:
                amount = 0.0
            desc = row['description_clean'][:55] if pd.notna(row['description_clean']) else ''
            print(f"  KES {amount:>8,.0f} | {desc}")
    
    print()
    
    # Spending summary
    print("=" * 80)
    print("SPENDING SUMMARY BY CATEGORY")
    print("=" * 80)
    
    spending_categories = df[df['Withdrawn'].notna()].copy()
    spending_categories['Withdrawn'] = pd.to_numeric(spending_categories['Withdrawn'], errors='coerce')
    
    cat_spending = spending_categories.groupby('category')['Withdrawn'].agg([
        ('Total', 'sum'),
        ('Count', 'count'),
        ('Average', 'mean')
    ]).sort_values('Total', ascending=False)
    
    print(f"\n{'Category':<30s} {'Total (KES)':>15s} {'Count':>8s} {'Avg (KES)':>12s}")
    print("-" * 80)
    
    total_spent = 0
    for category, row in cat_spending.head(15).iterrows():
        if category not in ['Income', 'Reversal', 'Cash Deposit']:
            total_spent += row['Total']
            print(f"{category:<30s} {row['Total']:>15,.2f} {int(row['Count']):>8,} {row['Average']:>12,.2f}")
    
    print("-" * 80)
    print(f"{'TOTAL SPENDING':<30s} {total_spent:>15,.2f}")
    print()
    
    # Save
    df.to_csv(output_csv, index=False)
    print(f"✅ Saved final categorized data: {output_csv}")
    print()
    
    print("=" * 80)
    print("STAGE 4 COMPLETE! 🎉")
    print("=" * 80)
    print()
    print("Your M-Pesa data is now fully categorized!")
    print(f"Categorization rate: {100 - final_uncat_pct:.1f}%")
    print(f"Total spending: KES {total_spent:,.2f}")
    print()
    
    return df


if __name__ == "__main__":
    # File paths
    INPUT_CSV = r"C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\stage3_with_categories.csv"
    OUTPUT_CSV = r"C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\stage4_final_categorized.csv"
    
    # Thresholds
    AMOUNT_THRESHOLD = 500.0  # KES
    RECURRING_THRESHOLD = 2   # Minimum occurrences
    
    # Run
    df = run_stage4(
        INPUT_CSV, 
        OUTPUT_CSV,
        amount_threshold=AMOUNT_THRESHOLD,
        recurring_threshold=RECURRING_THRESHOLD
    )
    
    print("Next steps:")
    print("  • Clean the CSV (remove empty/duplicate columns)")
    print("  • Analyze spending patterns")
    print("  • Create visualizations")

STAGE 4: SEND MONEY CATEGORIZATION

Preserving all Stage 2 & 3 improvements:
  ✅ Fuliza (LOAN) vs Fuliza payments
  ✅ Loan Repayment separate
  ✅ LOOP Payment → Income
  ✅ Data Bundles ≠ Airtime
  ✅ Cash Deposit ≠ Cash Withdrawal

RULES:
  1. Recurring (≥2) + Amount > 500.0 → Friends & Family
  2. Recurring (≥2) + Amount ≤ 500.0 → Merchant
  3. Non-recurring (any amount) → Merchant

📂 Loading: C:\Users\setla\Documents\Flatiron\PHASE5\Capstone\stage3_with_categories.csv
✓ Loaded 5,029 transactions

Current categorization:
  Uncategorized            : 1,329 ( 26.4%)
  Merchant                 :   724 ( 14.4%)
  M-Pesa Fees              :   606 ( 12.1%)
  Loans                    :   428 (  8.5%)
  Income                   :   411 (  8.2%)
  Airtime                  :   311 (  6.2%)
  Data Bundles             :   297 (  5.9%)
  Cash Withdrawal          :   275 (  5.5%)
  Loan Repayment           :   176 (  3.5%)
  Savings                  :   118 (  2.3%)

  Send Money (Uncategorized): 1,